# Autoencoder Experiment

In [1]:
import os

import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image

In [2]:

if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')


def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x

## Network Parameters

In [3]:
num_epochs = 100
batch_size = 128
learning_rate = 1e-3

In [5]:
img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = MNIST('./data', transform=img_transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

0.00B [00:00, ?B/s]

9.92MB [00:06, 1.58MB/s]                            


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz


0.00B [00:00, ?B/s]

32.8kB [00:01, 25.3kB/s]                            
0.00B [00:00, ?B/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz


1.65MB [00:08, 195kB/s]                            
0.00B [00:00, ?B/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz


8.19kB [00:00, 9.05kB/s]                   

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [6]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True), nn.Linear(64, 12), nn.ReLU(True), nn.Linear(12, 3))
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(True),
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True), nn.Linear(128, 28 * 28), nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [7]:
model = autoencoder().cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)

In [11]:
for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img).cuda()
        # ===================forward=====================
        output = model(img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss.item()))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, './mlp_img/image_{}.png'.format(epoch))

epoch [1/100], loss:0.1697
epoch [2/100], loss:0.1427
epoch [3/100], loss:0.1518
epoch [4/100], loss:0.1544
epoch [5/100], loss:0.1517
epoch [6/100], loss:0.1567
epoch [7/100], loss:0.1344
epoch [8/100], loss:0.1273
epoch [9/100], loss:0.1444
epoch [10/100], loss:0.1509
epoch [11/100], loss:0.1408
epoch [12/100], loss:0.1388
epoch [13/100], loss:0.1377
epoch [14/100], loss:0.1314
epoch [15/100], loss:0.1341
epoch [16/100], loss:0.1374
epoch [17/100], loss:0.1234
epoch [18/100], loss:0.1164
epoch [19/100], loss:0.1254
epoch [20/100], loss:0.1224
epoch [21/100], loss:0.1231
epoch [22/100], loss:0.1403
epoch [23/100], loss:0.1306
epoch [24/100], loss:0.1320
epoch [25/100], loss:0.1228
epoch [26/100], loss:0.1319
epoch [27/100], loss:0.1215
epoch [28/100], loss:0.1375
epoch [29/100], loss:0.1387
epoch [30/100], loss:0.1239
epoch [31/100], loss:0.1266
epoch [32/100], loss:0.1202
epoch [33/100], loss:0.1298
epoch [34/100], loss:0.1245
epoch [35/100], loss:0.1369
epoch [36/100], loss:0.1260
e

In [9]:
torch.save(model.state_dict(), './sim_autoencoder.pth')